# Roman Throughtput from Galsim

## Use the zero point to convert fluxes in magnitudes, check the ZP are corrects and merge dataframes with mags and redshifts

- author : Sylvie Dagoret-Campagne
- afflilation : IJCLab/IN2P3/CNRS
- creation date : 2025-03-11
- last update : 2025-03-11
- nersc python KERNEL : desc-python, or desc-python-bleed (better)

In [ ]:
import galsim
import os
import pandas as pd
import h5py

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

props = dict(boxstyle='round', facecolor="white", alpha=0.1)


import matplotlib.colors as colors
import matplotlib.cm as cmx
import seaborn as sns

In [ ]:
import galsim
import galsim.roman as roman
from  galsim.roman.roman_bandpass import getBandpasses

In [ ]:
plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
datadir = "/global/cfs/cdirs/descssim/imSim/skyCatalogs_v1.1.2"
throughputs_dir = "/global/cfs/cdirs/descssim/imSim/lsst/data/throughputs_aug_2021"
os.environ["THROUGHPUTS_DIR"] = throughputs_dir

### Select one HP 

In [ ]:
hpnum = 10552
data_name = f"skyCatalogs_v1.1.2 hp={hpnum}"
outputfile = f"magszgalaxies_lsstroman_skyCatalogs_v1.1.2_hp{hpnum}.parquet"

In [ ]:
fn_sed = f"galaxy_sed_{hpnum}.hdf5"
fn_galaxy = f"galaxy_{hpnum}.parquet"
fn_flux = f"galaxy_flux_{hpnum}.parquet"

## Read redshifts

In [ ]:
ffn_galaxy = os.path.join(datadir,fn_galaxy)
if not os.path.exists(ffn_galaxy) :
    raise Exception(f"File {ffn_galaxy} DOES NOT exists")
else:
    df_g = pd.read_parquet(ffn_galaxy)

In [ ]:
df_g.head()

In [ ]:
df_g = df_g[["galaxy_id","ra","dec","redshift"]]

In [ ]:
df_g

## Read flux file

In [ ]:
ffn_flux = os.path.join(datadir,fn_flux)
if not os.path.exists(ffn_flux):
    raise Exception(f"File {ffn_fluxy} DOES NOT exists")
else:
    df_f = pd.read_parquet(ffn_flux)

In [ ]:
df_f.head()

## Throughputs

### LSST Throughputs

In [ ]:
bands = "ugrizy"
lsst_ZP = {}
for band in bands:
    bp_file = os.path.join(os.environ['THROUGHPUTS_DIR'], 'baseline',
                           f'total_{band}.dat')
    lut = galsim.LookupTable.from_file(bp_file)
    bp = galsim.Bandpass(lut, wave_type='nm')
    bp = bp.truncate(relative_throughput=1e-3)
    bp = bp.thin()
    bp = bp.withZeropoint('AB')
    lsst_ZP[band]  = bp.zeropoint

In [ ]:
lsst_ZP

### Roman Throughputs

In [ ]:
#list_of_bands = ["F062","F087","F106","F129","F158","F184","F146","F213"]
list_of_bands = ["roman_flux_R062","roman_flux_Z087","roman_flux_Y106","roman_flux_J129","roman_flux_W146","roman_flux_H158","roman_flux_F184","roman_flux_K213"]
Nb = len(list_of_bands)
colors = sns.color_palette("Spectral_r", Nb)
colors

In [ ]:
roman_bandpasses = roman.getBandpasses(AB_zeropoint=True)

In [ ]:
#filters = [filter_name for filter_name in roman_filters if filter_name[0] in use_filters]
roman_filters_names = [filter_name for filter_name in roman_bandpasses]

In [ ]:
roman_filters_names

In [ ]:
Nf = len(roman_filters_names)
colors = sns.color_palette("Spectral_r", Nb)
colors

In [ ]:
roman_ZP = {}
for filt_name in roman_filters_names:
    roman_ZP[filt_name]  = roman_bandpasses[filt_name].zeropoint

In [ ]:
roman_ZP

In [ ]:
roman_bandpasses = galsim.roman.getBandpasses()

### Convertion in magnitudes
- To convert from flux (which has units of #/photons/cm^2/s) in the skyCatalogs files to magnitude, do :

      mag = zp - 2.5*log10(flux)


  

In [ ]:
flux_cols = df_f.columns[1:]
flux_cols

In [ ]:
df_m = pd.DataFrame()
df_m = df_f[["galaxy_id"]]

In [ ]:
df_m.head()

In [ ]:
for flux_col in flux_cols:
    split_colname = flux_col.split("_")
    instrum =   split_colname[0]
    band = split_colname[-1]
    mag_colname = f"{instrum}_mag_{band}"
    print(mag_colname)
    if instrum == "lsst":
        df_m.loc[:,mag_colname] = lsst_ZP[band] - 2.5*(df_f[flux_col].apply(np.log10))
    elif instrum == "roman":
        df_m.loc[:,mag_colname] = roman_ZP[band] - 2.5*(df_f[flux_col].apply(np.log10))

In [ ]:
df_m

In [ ]:
colors = sns.color_palette("hls", 3)

fig,axs = plt.subplots(1,3,figsize=(16,5),layout="constrained")
ax1,ax2,ax3 = axs.flatten()


df_m.plot.scatter(x="lsst_mag_r",y="roman_mag_R062",ax=ax1,marker=".",color=colors[0])
ax1.set_aspect('equal')

df_m.plot.scatter(x="lsst_mag_z",y="roman_mag_Z087",ax=ax2,marker=".",color=colors[1])
ax2.set_aspect('equal')


df_m.plot.scatter(x="lsst_mag_y",y="roman_mag_Y106",ax=ax3,marker=".",color=colors[2])
ax3.set_aspect('equal')

plt.suptitle("Comparison magnitudes roman vs LSST")

plt.show()

In [ ]:
df_m.head()

In [ ]:
df_g.head()

In [ ]:
df_mag = df_m.merge(df_g, on="galaxy_id")

In [ ]:
df_mag

## Save output file

In [ ]:
df_mag.to_parquet(outputfile)